In [1]:
import torch
import torch.nn as nn
import networkx as nx
import matplotlib.pyplot as plt
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork, DynapcnnNetworkGraph
import sinabs as snb

In [2]:
torch.manual_seed(0)

In [3]:
channels = 1
height = 28
width = 28

input_shape = (channels, height, width)

## Network Module (pure Pytorch)

In [4]:
class ANN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.con1 = nn.Conv2d(1, 20, 5, 1, bias=False)
        self.rel1 = nn.ReLU()
        self.pool1 = nn.AvgPool2d(2,2)

        self.conv2 = nn.Conv2d(20, 32, 5, 1, bias=False)
        self.rel2 = nn.ReLU()
        self.pool2 = nn.AvgPool2d(2,2)

        self.conv3 = nn.Conv2d(32, 128, 3, 1, bias=False)
        self.rel3 = nn.ReLU()
        self.pool3 = nn.AvgPool2d(2,2)

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(128, 500, bias=False)
        self.rel4 = nn.ReLU()
        self.fc2 = nn.Linear(500, 10, bias=False)

    def forward(self, x):
        
        con1_out = self.con1(x)
        rel1_out = self.rel1(con1_out)
        pool1_out = self.pool1(rel1_out)

        conv2_out = self.conv2(pool1_out)
        rel2_out = self.rel2(conv2_out)
        pool2_out = self.pool2(rel2_out)

        conv3_out = self.conv3(pool2_out)
        rel3_out = self.rel3(conv3_out)
        pool3_out = self.pool3(rel3_out)

        flat_out = self.flat(pool3_out)
        
        fc1_out = self.fc1(flat_out)
        rel4_out = self.rel4(fc1_out)
        fc2_out = self.fc2(rel4_out)

        return fc2_out

In [5]:
ann = ANN()

## Sinabs Model

In [6]:
sinabs_model = from_model(ann, add_spiking_output=True, batch_size=1)

## DynapCNN Model

In [7]:
hw_model = DynapcnnNetworkGraph(
    sinabs_model,
    discretize=True,
    input_shape=input_shape,
    use_jit_tracer=False
)

In [8]:
# hw_model.to(device="speck2edevkit:0")

In [9]:
print(hw_model)


layer index: 0
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(635.), min_v_mem=Parameter containing:
  tensor(-635.), batch_size=1, num_timesteps=-1)
  (pool_layer): SumPool2d(norm_type=1, kernel_size=(2, 2), stride=None, ceil_mode=False)
)
layer destinations: [1]
assigned core: -1

layer index: 1
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(11361.), min_v_mem=Parameter containing:
  tensor(-11361.), batch_size=1, num_timesteps=-1)
  (pool_layer): SumPool2d(norm_type=1, kernel_size=(2, 2), stride=None, ceil_mode=False)
)
layer destinations: [2]
assigned core: -1

layer index: 2
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_thre